# JPMC InsideSherpa - Module 2 
Pedro Teixeira - O734271

## Challenge

> Input a name on the command prompt, match it with the above list and return a “Hit” or “No hit” based on a 75% match.
>
> ### Expected technical output:
>	
> A Reusable Python/Java Classes which can used to Identify Payee is Matching against given list of Sanctions (List which contains Name of Individuals, Organizations and Countries provided as part of CSV file) and returns with Hit / No with Percentages,
>
> ### Expected user action/input
>
> Payee Information through command line argument or build a simple rest endpoint in Python/Java is real bonus
>
> ### Tools
>
> Python with Jupiter notebook setup or Java Implementation + Rest endpoint is real bonus


## Defining a match

* What does a 75% match mean?

It is not clear exactly what the task means by a 75% match. If we assume the simplest possible solution it would be using the hamming distance. However, the Hamming distance is not appropriate to compare string of different lengths.

The US Department of the Treasury [seems to use Jaro-Winkler](https://www.treasury.gov/resource-center/sanctions/SDN-List/Pages/fuzzy_logic.aspx) distance/similarity measure. We will try to define something that we can implement, as Jaro-Winkler seems to be a little to complex, even though good libraries are available.

After some research on string similarity quickly brings up lots of options: phonetically based algorithms, fuzzy logic, etc. (see https://stackabuse.com/levenshtein-distance-and-text-similarity-in-python/). 

The [Levenshtein distance](https://en.wikipedia.org/wiki/Levenshtein_distance) seemed to be the most appropriate as it represents the minimum number of edits to transform one of the strings into the other. It is also not too complex and can be solved with dynammic programming. I would even claim that it goes well with the context of someone trying to escape the sanction screening, in the way that humans would think of how to modify our name. Phonetic algorithms would be the next step.

### Properties of Levenshtein distance

From the Wikipedia page:

> The Levenshtein distance has several simple upper and lower bounds. These include:
> * It is at least the difference of the sizes of the two strings.
* **It is at most the length of the longer string.**
* It is zero if and only if the strings are equal.
* If the strings are the same size, the Hamming distance is an upper bound on the Levenshtein distance.
* The Levenshtein distance between two strings is no greater than the sum of their Levenshtein distances from a third string (triangle inequality).

[We can use this to derive a **basic** similarity score](https://stackoverflow.com/questions/6087281/similarity-score-levenshtein):

`similarity = 1 - (levenshtein_distance / n)`  where n is the length of the longest string.

Since the scores here tend to be pretty low, we can apply logarithmic scaling and define it as:

`similarity = 1 / (1 - log(1 - (levenshtein_distance / n))) `

or

$ similarity = \frac{1}{1-log(score))} $,

where $ score = 1 - \frac{levenshtein\_distance}{n} $

$log$ could be the logarithm of any basse in the above formulas. The base can be varied to adjust the desired scaling given appropriate data on how it is performing. Some quick testing reveals that base 20 seems to give similar scores to the US DoT tool, but further investigation would be needed to confirm this. Nevertheless, we will use base 20 for this exercise.

This will be our definition of match for this task:

> 2 strings match iff this logarithmic similarity measure is greater than the set threshold (75%)


## Utility code

In [ ]:
# Initialize data
names = [
         "Kristopher Doe",
         "Iceland",
         "Real Arctic Line"
]

In [4]:
# Install Levenshtein distance library
!pip install textdistance

## Solution

### Building blocks

#### With library code

In [8]:
from textdistance import levenshtein

test1 = levenshtein("text", "test")
test2 = levenshtein("potato", "boat")

print("Similarity 'text'-'test': ", test1)
print("Similarity 'potato'-'boat': ", test2)

Similarity 'text'-'test':  1
Similarity 'potato'-'boat':  3


#### Own implementation


In [10]:
import numpy as np

In [11]:
def distance(str1: str, str2: str):
  """
  Naive Implementation of Levenshtein Distance
  
  According to this video: 
  https://www.youtube.com/watch?time_continue=584&v=We3YDTzNXEk&feature=emb_logo
  """
  
  # ignore case
  str1 = str1.lower()
  str2 = str2.lower()

  m = len(str1)
  n = len(str2)
  dists = np.zeros((m+1, n+1))
  dists[0] = np.arange(n+1)
  dists[:,0] = np.arange(m+1)

  for i in range(1, m+1):
    for j in range(1, n+1):
      if str1[i-1] == str2[j-1]:
        dists[i,j] = dists[i-1,j-1]
      else:
        add = dists[i, j-1]
        delete = dists[i-1, j]
        replace = dists[i-1, j-1]
        
        dists[i,j] = min(add, delete, replace) + 1
      
  return int(dists[m,n]), m, n

In [12]:
def similarity(str1: str, str2: str):
  """
  Computes the basic similarity score according to the above discussion
  """
  d, l1, l2 = distance(str1, str2)
  return 1 - (d / max(l1, l2))

In [13]:
from math import log

def log_similarity(str1: str, str2: str):
  """
  Computes the logarithmic similarity score according to the above discussion
  """
  d, l1, l2 = distance(str1, str2)
  score = 1 - (d / max(l1, l2))

  if score == 0:
    return 0

  return 1 / (1 - log(score, 20))

#### Some basic test cases just o make sure things look correct

In [14]:
# Testing basic examples
test1, _, _ = distance("test", "text")
test2, _, _ = distance("potato", "boat")

print("Distance 'text'-'test': ", test1)
print("Distance 'potato'-'boat': ", test2)

assert test1 == 1
assert test2 == 3

Distance 'text'-'test':  1
Distance 'potato'-'boat':  3


In [16]:
score = similarity("potato", "boat")

assert score == 0.5 # 3 / 6

### Putting it all together

In [19]:
# This will require some workaround if the sanctions list does not fit in
# memory but it should be easily solved by batch loading it and persisting the 
# results every few batches if needed

def screen_name(search_name: str, sanctions_list: str, threshold=0.75, similarity_function=log_similarity):
    """
    Screens the given name against the given list of sanctioned names,
    returning a list of names together with their similarity scores if 
    those are at least as large as the threshold.    
    """
    sanctioned_names = None
    with open(sanctions_list, 'r') as f:
        # need to remove newline char at end of each name
        # there should be no empty lines in the input file
        sanctioned_names = [x[:-1] for x in f.readlines()]

    hits = []
    for sanctioned_name in sanctioned_names:
        score = similarity_function(sanctioned_name, search_name)
        if score >= threshold:
            hits.append((sanctioned_name, round(score * 100)))

    # sort the list of hits before returning
    hits.sort(key=lambda x: x[1], reverse=True)

    return hits


## Testing

To reveal the accuracy of our efforts, let us test them first on some basic examples and then on the real SDN (Specially Designated Nationals) dataset, downloaded from the US Department of the Treasury, available [here](https://www.treasury.gov/ofac/downloads/sdn.csv).

In [21]:
# Basic test examples
sanctions_list = "/content/drive/My Drive/Colab Notebooks/sanctions.txt" # Adjust this according to the environment
search_name = "Kristophre Toe"

screen_name(search_name, sanctions_list)


[('Kristopher Doe', 93)]

### Cleaning up the data

In [22]:
import pandas as pd
import numpy as np

In [23]:
# Clean up the data to just extract the names into a file
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/sdn.csv", header=None, skipfooter=1)

sdn_names = df[1].to_list()

with open('sdn_names.txt', 'w') as f:
  for name in sdn_names:
    f.write(name + '\n')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  


In [28]:
sanctions_list = "sdn_names.txt"
search_name = "fly dargon"

screen_name(search_name, sanctions_list)

[('FLYING DRAGON', 86),
 ('RI, Man Gon', 79),
 ('AL FURQAN', 77),
 ('FAN PARDAZAN', 77),
 ('GAYE, Haroun', 77),
 ('YARAN', 77),
 ('AZARGOUN', 77),
 ('FAXON', 77),
 ('PDVSA CARDON', 77),
 ('BALA, Haradin', 76),
 ('FADUL, Farhub', 76),
 ('FARS SARPANAH', 76)]

## REST API

As Jupyter seems to be more appropriate for investigative work, the REST API is included as aseparate file.